# Zillow Property Search
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1afLUQVYLIhv0vDB-GNuYgxfZrJhTzSM5?usp=sharing)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 22nd Jan 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

## <font color="blue">Functions</font>

In [2]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

## <font color="blue">Locals & Constants</font>

In [3]:
############
# OPTIONAL #
############

# mount drive
drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Data/input/" # optional

Mounted at /content/drive


In [4]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

### <font color="green">1. Property Listings For Sale</font>

In [5]:
# zillow search url
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

# get listings
listing_response = get_listings(api_key, listing_url)

In [6]:
# view all keys
listing_response.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [7]:
# check if request is successful
listing_response.json()["is_success"]

True

In [8]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 232


In [9]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 223
Number of columns: 72


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,availabilityDate,latLong.latitude,latLong.longitude,variableData.type,variableData.text,hdpData.homeInfo.zpid,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,variableData,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.listing_sub_type.is_forAuction,hdpData.homeInfo.unit
0,45070261,"$550,000",$550K,3,3.0,1612,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/90b5df05526...,True,False,,False,/homedetails/6208-W-Thorpe-St-Tampa-FL-33611/4...,ForSale,For Sale (Broker),True,False,False,--,False,False,None,27.877790,-82.501590,3D_HOME,3D Tour,45070261,33611,Tampa,FL,27.877790,-82.501590,550000.0,3.0,3.0,1612.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,463200.0,2949.0,True,False,False,FOR_SALE,550000.0,True,False,False,USD,USA,235412.0,6250.000000,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,99616959,"$350,000",$350K,3,2.0,1630,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/ab2d93a3f55...,True,False,,False,/homedetails/3611-Potter-St-Tampa-FL-33605/996...,ForSale,For Sale (Broker),False,False,False,--,False,False,None,27.976728,-82.432884,PRICE_REDUCTION,"$15,000 (Feb 4)",99616959,33605,Tampa,FL,27.976728,-82.432884,350000.0,2.0,3.0,1630.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,336792.0,2414.0,True,False,False,FOR_SALE,350000.0,True,False,False,USD,USA,35532.0,5076.000000,sqft,1.675498e+12,"$15,000 (Feb 4)",-15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82046466,"$265,000",$265K,3,2.0,1290,FOR_SALE,New construction,False,False,False,https://photos.zillowstatic.com/fp/813010716aa...,True,False,NEW_CONSTRUCTION,False,/homedetails/8206-N-14th-St-Tampa-FL-33604/820...,ForSale,New Construction,False,False,False,--,False,False,None,28.022974,-82.444450,NaN,NaN,82046466,33604,Tampa,FL,28.022974,-82.444450,265000.0,2.0,3.0,1290.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,NaN,NaN,NaN,False,False,FOR_SALE,265000.0,True,False,False,USD,USA,21250.0,5000.000000,sqft,NaN,NaN,NaN,NaN,True,BUILDER_SPEC,NaN,NaN,NaN,NaN,NaN,NaN
3,89745040,"$299,000",$299K,5,3.0,1624,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/671ef8d50ef...,True,False,,False,/homedetails/7706-N-Riverdale-Ave-Tampa-FL-336...,ForSale,For Sale (Broker),False,False,False,--,False,False,None,28.018438,-82.456510,OPEN_HOUSE,Open: Sun. 12:30-3pm,89745040,33604,Tampa,FL,28.018438,-82.456510,299000.0,3.0,5.0,1624.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,294400.0,2422.0,True,False,False,FOR_SALE,299000.0,True,False,False,USD,USA,180493.0,4889.000000,sqft,1.676534e+12,"$16,000 (Feb 16)",-16000.0,NaN,NaN,NaN,True,Sun. 12:30-3pm,"[{'open_hou

In [10]:
# view price
px.histogram(df_listings, x="hdpData.homeInfo.price", title="Sales Price Histogram")

In [11]:
# view zestimate
px.histogram(df_listings, x="hdpData.homeInfo.zestimate", title="Zestimate Histogram")

In [12]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">2. Rental Property Listings For Rent</font>

In [13]:
# zillow search url
rent_listing_url = "https://www.zillow.com/tampa-fl/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.69632304150392%2C%22east%22%3A-82.2122379584961%2C%22south%22%3A27.743327003864252%2C%22north%22%3A28.2453440906427%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priorityscore%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22beds%22%3A%7B%22min%22%3A3%2C%22max%22%3A3%7D%2C%22baths%22%3A%7B%22min%22%3A2%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D"

# get listings
rent_listing_response = get_listings(api_key, rent_listing_url)

In [14]:
# view all keys
print(rent_listing_response.json().keys())

# check if request is successful
print("Request success:", rent_listing_response.json()["is_success"])

# view count of properies returned in request
num_of_properties = rent_listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

dict_keys(['is_success', 'data', 'message'])
Request success: True
Count of properties: 183


In [15]:
# view all listings
df_rent_listings = pd.json_normalize(rent_listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_rent_listings))
print("Number of columns:", len(df_rent_listings.columns))
df_rent_listings

Number of rows: 180
Number of columns: 67


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,availabilityDate,latLong.latitude,latLong.longitude,variableData.type,variableData.text,variableData.data.isFresh,hdpData.homeInfo.zpid,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.unit,streetViewMetadataURL,streetViewURL
0,45098471,"$3,200/mo",$3.2K,3,2.0,1355.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/5d4b1ac0f01...,True,False,,False,/homedetails/1305-E-Osborne-Ave-Tampa-FL-33603...,ForRent,For Rent,False,False,False,--,False,True,None,27.988577,-82.44454,TIME_ON_INFO,3 days ago,False,45098471,33603,Tampa,FL,27.988577,-82.44454,3200.0,2.0,3.0,1355.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,388070.0,2314.0,False,False,FOR_RENT,3200.0,True,False,False,USD,USA,89765.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45082321,"$1,950/mo",$2.0K,3,2.0,1152.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/07a7f54691f...,True,False,,False,/homedetails/8311-N-Semmes-St-Tampa-FL-33604/4...,ForRent,For Rent,False,False,False,--,False,True,2023-02-16 00:00:00,28.024832,-82.44146,TIME_ON_INFO,2 days ago,False,45082321,33604,Tampa,FL,28.024832,-82.44146,1950.0,2.0,3.0,1152.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,NaN,2019.0,False,False,FOR_RENT,1950.0,True,False,False,USD,USA,118240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,44922139,"$2,600/mo",$2.6K,3,2.0,1700.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/06751c3571c...,True,False,,False,/homedetails/8326-Torrington-Ave-Tampa-FL-3364...,ForRent,For Rent,False,False,False,--,False,True,2023-03-14 00:00:00,28.139587,-82.37024,TIME_ON_INFO,4 days ago,False,44922139,33647,Tampa,FL,28.139587,-82.37024,2600.0,2.0,3.0,1700.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,414000.0,2396.0,False,False,FOR_RENT,2600.0,True,False,False,USD,USA,222198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55026435,"$2,295/mo",$2.3K,3,2.0,1474.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/8e43d64e5fa...,True,False,,False,/homedetails/19224-Wood-Sage-Dr-Tampa-FL-33647...,ForRent,For Rent,False,False,False,--,False,True,2023-04-13 00:00:00,28.154194,-82.36617,TIME_ON_INFO,2 days ago,False,55026435,33647,Tampa,FL,28.154194,-82.36617,2295.0,2.0,3.0,1474.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,343900.0,2300.0,False,False,FOR_RENT,2295.0,True,False,False,USD,USA,175164.0,1.676621e+12,$5 (Feb 17),-5.0,NaN,NaN,NaN,NaN,NaN
4,45099882,"$2,100/mo",$2.1K,3,2.0,1307.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/696116c38d6...,True,False,,False,/homedetails/3711-N-31st-St-Tampa-FL-33610/450...,ForRent,For Rent,False,False,False,--,False,True,2023-02-25 00:00:00,27.979162,-82.42528,TIME_ON_INFO,15 days ago,False,45099882,33610,Tampa,FL,27.979162,-82.42528,2100.0,2.0,3.0,1307.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,F

In [16]:
# view price
px.box(df_rent_listings, x="hdpData.homeInfo.price", title="Rental Price Box Plot")

In [17]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">3. Property Details</font>

In [18]:
# property unique id
zpid = "44905318"

# get property detail
prop_detail_response = get_property_detail(api_key, zpid)

In [19]:
# view all keys
print(prop_detail_response.json().keys())

# check if request is successful
print("Request success:", prop_detail_response.json()["is_success"])

dict_keys(['is_success', 'data', 'message'])
Request success: True


In [20]:
# view property details
df_prop = pd.json_normalize(prop_detail_response.json()['data'])
print("Number of rows:", len(df_prop))
print("Number of columns:", len(df_prop.columns))
df_prop

Number of rows: 1
Number of columns: 602


,listingDataSource,zpid,city,state,homeStatus,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,bedrooms,bathrooms,price,yearBuilt,streetAddress,zipcode,isCurrentSignedInUserVerifiedOwner,regionString,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,isRentalListingOffMarket,hdpUrl,nearbyCities,nearbyNeighborhoods,country,nearbyZipcodes,abbreviatedAddress,neighborhoodRegion,building,isUndisclosedAddress,boroughId,providerListingID,neighborhoodSearchUrl,boroughSearchUrl,communityUrl,isPremierBuilder,isZillowOwned,homeType,currency,listPriceLow,livingArea,livingAreaValue,zestimate,newConstructionType,zestimateLowPercent,zestimateHighPercent,rentZestimate,restimateLowPercent,restimateHighPercent,schools,homeValues,nearbyHomes,countyFIPS,parcelId,taxHistory,priceHistory,comps,description,whatILove,contingentListingType,timeOnZillow,pageViewCount,favoriteCount,daysOnZillow,latitude,longitude,openHouseSchedule,desktopWebHdpImageLink,brokerageName,timeZone,pals,listingAccountUserId,homeInsights,sellingSoon,listingProvider,isIncomeRestricted,brokerId,ssid,monthlyHoaFee,propertyTaxRate,hiResImageLink,hdpTypeDimension,mlsid,propertyTypeDimension,mediumImageLink,enhancedBrokerImageUrl,responsivePhotos,buildingId,virtualTourUrl,hasApprovedThirdPartyVirtualTourUrl,photoCount,livingAreaUnits,lotSize,lotAreaValue,lotAreaUnits,postingProductType,marketingName,richMedia,cityId,stateId,zipPlusFour,numberOfUnitsTotal,foreclosureDefaultFilingDate,foreclosureAuctionFilingDate,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureLoanAmount,foreclosurePriorSaleDate,foreclosurePriorSaleAmount,foreclosureBalanceReportingDate,foreclosurePastDueBalance,foreclosureUnpaidBalance,foreclosureAuctionTime,foreclosureAuctionDescription,foreclosureAuctionCity,foreclosureAuctionLocation,foreclosureDate,foreclosureAmount,foreclosingBank,foreclosureJudicialType,datePostedString,foreclosureMoreInfo,hasBadGeocode,streetViewMetadataUrlMediaWallLatLong,streetViewMetadataUrlMediaWallAddress,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewServiceUrl,postingUrl,hasPublicVideo,primaryPublicVideo,richMediaVideos,photos,tourViewCount,listingAccount,listingFeedID,livingAreaUnitsShort,priceChange,priceChangeDate,priceChangeDateString,hideZestimate,comingSoonOnMarketDate,isPreforeclosureAuction,lastSoldPrice,isHousingConnector,responsivePhotosOriginalRatio,streetViewMetadataUrlMapLightboxAddress,thumb,isRecentStatusChange,isNonOwnerOccupied,county,isFeatured,rentalApplicationsAcceptedType,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,address.streetAddress,address.city,address.state,address.zipcode,address.neighborhood,address.community,address.subdivision,listing_sub_type.is_newHome,listing_sub_type.is_FSBO,listing_sub_type.is_FSBA,listing_sub_type.is_foreclosure,listing_sub_type.is_bankOwned,listing_sub_type.is_forAuction,listing_sub_type.is_comingSoon,listing_sub_type.is_pending,listing_sub_type.is_openHouse,citySearchUrl.text,citySearchUrl.path,zipcodeSearchUrl.path,apartmentsForRentInZipcodeSearchUrl.path,housesForRentInZipcodeSearchUrl.path,stateSearchUrl.path,countySearchUrl.text,countySearchUrl.path,adTargets.aamgnrc1,adTargets.bd,adTargets.fsbid,adTargets.city,adTargets.proptp,adTargets.pid,adTargets.lot,adTargets.zestibuck,adTargets.listtp,adTargets.sqftrange,adTargets.price,adTargets.sqft,adTargets.state,adTargets.mlong,adTargets.cnty,adTargets.prange,adTargets.zip,adTargets.mlat,adTargets.zusr,adTargets.price_band,adTargets.yrblt,adTargets.zestimate,adTargets.premieragent,adTargets.dma,adTargets.guid,adTargets.ssid,resoFacts.accessibilityFeatures,resoFacts.additionalFeeInfo,resoFacts.associationFee,resoFacts.associationAmenities,resoFacts.associationFee2,resoFacts.associationFeeIncludes,resoFacts.associationName,resoFacts.associationName2,resoFacts.associationPhone

In [21]:
print("Street address:", df_prop["streetAddress"].iloc[0])
print("City:", df_prop["city"].iloc[0])
print("State:", df_prop["state"].iloc[0])
print("Home status:", df_prop["homeStatus"].iloc[0])
print("Bedrooms:", df_prop["bedrooms"].iloc[0])
print("Bathrooms:", df_prop["bathrooms"].iloc[0])
print("Year built:", df_prop["yearBuilt"].iloc[0])
print("Zestimate: ${:,}".format(df_prop["zestimate"].iloc[0]))
print("Rent zestimate: ${:,}".format(df_prop["rentZestimate"].iloc[0]))
print("Description:")
df_prop["description"].iloc[0]

Street address: 12315 N Oakleaf Ave
City: Tampa
State: FL
Home status: PENDING
Bedrooms: 3
Bathrooms: 2
Year built: 1977
Zestimate: $323,900
Rent zestimate: $2,154
Description:


'Forest Hills Charmer!! This home has been loved and cared for by one owner for over 40 years. It offers 3 bedrooms and 2 full bathrooms. The oversized carport has a large enclosed storage area with double door access. The kitchen has been updated and offers plenty of cabinets and an efficient work space area. New Vinyl Wood Plank flooring is in all the right places. The family room is very open and light and bright. French doors lead to your beautiful lanai enclosure, which is a great place to unwind, entertain or gather with friends and family. Beyond the lanai enjoy a fabulous lushly landscaped fenced back yard. The master sits in the back of the house and has a full bath. It  also opens to the outside space. When you hear Location, Location Location this is it! Really the heart of Tampa! Downtown is 20 minutes away, Roy Haynes Park and Rec Center is a 5 minute walk and Babe Zaharias Golf and Country Club is around the corner. Close to shopping, restaurants, and world renowned Flori

#### Comparables

In [22]:
# view comparables
df_c = pd.DataFrame(df_prop["comps"].iloc[0])
df_c

,zpid,miniCardPhotos,price,currency,bedrooms,bathrooms,livingArea,livingAreaValue,livingAreaUnits,livingAreaUnitsShort,listingMetadata,lotSize,lotAreaValue,lotAreaUnits,address,formattedChip,latitude,longitude,homeStatus,homeType,hdpUrl,listing_sub_type,providerListingID,attributionInfo,isPremierBuilder,state,isZillowOwned,newConstructionType
0,44894167,[{'url': 'https://photos.zillowstatic.com/fp/3...,330000,USD,3,2,1290,1290,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},10744,10744,Square Feet,"{'streetAddress': '12706 Linda Dr', 'city': 'T...","{'location': [{'fullValue': '12706 Linda Dr'},...",28.062586,-82.462654,FOR_SALE,SINGLE_FAMILY,/homedetails/12706-Linda-Dr-Tampa-FL-33612/448...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3428470', 'mlsName': None, 'provid...",False,FL,False,None
1,45074037,[{'url': 'https://photos.zillowstatic.com/fp/d...,319000,USD,3,2,1094,1094,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},6000,6000,Square Feet,"{'streetAddress': '10913 N 22nd St', 'city': '...",{'location': [{'fullValue': '10913 N 22nd St'}...,28.048338,-82.434370,FOR_SALE,SINGLE_FAMILY,/homedetails/10913-N-22nd-St-Tampa-FL-33612/45...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'U8190363', 'mlsName': None, 'provid...",False,FL,False,None
2,44905268,[{'url': 'https://photos.zillowstatic.com/fp/7...,330000,USD,3,1,1060,1060,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},7345,7345,Square Feet,"{'streetAddress': '12304 Woodleigh Ave', 'city...",{'location': [{'fullValue': '12304 Woodleigh A...,28.059185,-82.479805,FOR_SALE,SINGLE_FAMILY,/homedetails/12304-Woodleigh-Ave-Tampa-FL-3361...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3418673', 'mlsName': None, 'provid...",False,FL,False,None
3,44905298,[{'url': 'https://photos.zillowstatic.com/fp/9...,459900,USD,4,3,1828,1828,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},6000,6000,Square Feet,"{'streetAddress': '12330 N Rome Ave', 'city': ...",{'location': [{'fullValue': '12330 N Rome Ave'...,28.060783,-82.476074,FOR_SALE,SINGLE_FAMILY,/homedetails/12330-N-Rome-Ave-Tampa-FL-33612/4...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3429109', 'mlsName': None, 'provid...",False,FL,False,None
4,44905143,[{'url': 'https://photos.zillowstatic.com/fp/3...,385000,USD,3,2,1024,1024,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},9375,9375,Square Feet,"{'streetAddress': '12909 N Albany Ave', 'city'...",{'location': [{'fullValue': '12909 N Albany Av...,28.063860,-82.478870,FOR_SALE,SINGLE_FAMILY,/homedetails/12909-N-Albany-Ave-Tampa-FL-33612...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'U8187845', 'mlsName': None, 'provid...",False,FL,False,None
5,82038541,[{'url': 'https://maps.googleapis.com/maps/api...,338000,USD,4,2,1392,1392,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': False},6534,6534,Square Feet,"{'streetAddress': '1109 E Seneca Ave', 'city':...",{'location': [{'fullValue': '1109 E Seneca Ave...,28.048548,-82.448470,FOR_SALE,SINGLE_FAMILY,/homedetails/1109-E-Seneca-Ave-Tampa-FL-33612/...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': '', 'mlsName': None, 'providerLogo':...",False,FL,False,None
6,2059245099,[{'url': 'https://photos.zillowstatic.com/fp/1...,250000,USD,3,2,1306,1306,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},4,4,Square Feet,{'streetAddress': '12269 Armenia Gables Cir #1...,{'location': [{'fullValue': '12269 Armenia Gab...,28.059134,-82.481080,FOR_SALE,CONDO,/homedetails/12269-Armenia-Gables-Cir-12269-Ta...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3428280', 'mlsName': None, 'provid...",False,FL,False,None
7,45084262,[{'url': 'https://photos.zillowstatic.com/fp/3...,331000,USD,3,3,1340,1340,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},10125,10125,Square Feet,"{'streetAd

#### Tax History

In [23]:
# view tax history
df_tax_hist = pd.DataFrame(df_prop["taxHistory"].iloc[0])
df_tax_hist

,time,taxPaid,taxIncreaseRate,value,valueIncreaseRate
0,1613682354733,874.77,0.080857,77389,0.013994
1,1582059954733,809.33,0.087488,76321,0.023001
2,1550523954733,744.22,0.000000,74605,0.018999
3,1518987954733,744.22,0.045400,73214,-0.363035
4,1487451954733,711.90,0.009000,114942,0.636581
5,1455829554733,705.55,0.007569,70233,0.006997
6,1424293554733,700.25,0.006193,69745,0.008007
7,1392757554733,695.94,0.000000,69191,0.012349
8,1361221554733,695.94,-0.030360,68347,0.015240
9,1329599154733,717.73,0.019720,67321,0.030003


#### Price History

In [24]:
# view price history
df_price_hist = pd.DataFrame(df_prop["priceHistory"].iloc[0])
df_price_hist

,date,time,price,pricePerSquareFoot,priceChangeRate,event,source,buyerAgent,sellerAgent,showCountyLink,postingIsRental,attributeSource
0,2023-01-23,1674432000000,330000,263,0,Pending sale,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'U8187986', 'infoString2': 'St..."
1,2023-01-20,1674172800000,330000,263,0,Listed for sale,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'U8187986', 'infoString2': 'St..."


### <font color="green">4. Locate ZPID by Address</font>

In [25]:
street = "12315 N Oakleaf Ave"
city = "Tampa"
state = "FL"
# zip_code = "33612" #[optional]

zpid_response = get_zpid(api_key, street, city, state, zip_code=None)

In [26]:
zpid_response.json()["is_success"]

True

In [27]:
# view response, top possible zpids
pd.DataFrame(zpid_response.json()["data"])

,zpid,url
0,44905318,https://www.zillow.com/homedetails/12315-N-Oak...
1,44905317,https://www.zillow.com/homedetails/12311-N-Oak...
2,44905319,https://www.zillow.com/homedetails/12319-N-Oak...
3,44905324,https://www.zillow.com/homedetails/12315-Woodl...
4,44905320,https://www.zillow.com/homedetails/1714-Round-...
5,250619234,https://www.zillow.com/homedetails/12317-Woodl...


In [28]:
print("Zpid:", zpid_response.json()["data"][0]["zpid"])

Zpid: 44905318


# End Notebook